In [ ]:
import pandas as pd, numpy as np
import os

In [ ]:
os.chdir('C:\\Users\\umair\\Desktop\\uber')

In [ ]:
df = pd.read_csv('uber_data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [ ]:
df.info()

In [ ]:
# droping duplicates and removing inconsistent indexesdue to the deletion process
df = df.drop_duplicates().reset_index(drop=True)
# to start index from 1 instead of zero
df.index = np.arange(1, len(df) + 1) 

In [ ]:
# creating a trip_id column 
# having values equals to the index values
df['trip_id'] = df.index

In [39]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_id
1,1,2016-03-01,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35,1
2,1,2016-03-01,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35,2
3,2,2016-03-01,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80,3
4,2,2016-03-01,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62,4
5,2,2016-03-01,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80,5


## Creating Dimension Tables

### 1. datetime_dim

In [ ]:
#creating datetime dimension table
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']]

# adding column in the table
datetime_dim['datetime_id'] = datetime_dim.index
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday
# -- below columns could have been in a seperate table

datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

#rearranging column order
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 
                             'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 
                             'drop_month', 'drop_year', 'drop_weekday']]

datetime_dim.head()

### 2. pickup_location_dim

In [ ]:
pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']]
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']] 
pickup_location_dim

### 3. dropoff_location_dim

In [ ]:
dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']]
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]
dropoff_location_dim

### 4. passenger_count_dim

In [ ]:
# creating passenger_count dimension table
passenger_count_dim = df[['passenger_count']]
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
# rearranging the columns
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]
passenger_count_dim

### 5. trip_distance_dim

In [ ]:
#creating trip_distance dimension table
trip_distance_dim = df[['trip_distance']]
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
#rearranging the columns
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]
trip_distance_dim

### 6. rate_code_dim

In [ ]:
#creating a dictionary to map the rate_code_names 
#with rate_code_number
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

#creating the rate_code dimension table now
rate_code_dim = df[['RatecodeID']]
rate_code_dim['rate_code_id'] = rate_code_dim.index
#using map function to map the rate_code_name to the rate_code_type from a dictionary
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
#rearranging the columns as required
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]
rate_code_dim

### 7. payment_type_dim

In [ ]:
# creating payment_type dictionary to map with the payment_type_id
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
#creating the dimension table for payment_type
payment_type_dim = df[['payment_type']]
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
#rearranging the columns 
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]
payment_type_dim

### FACT TABLE

In [42]:
fact_table = fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,1,1,1,1,1,N,1,1,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,2,1,2,2,2,2,N,2,2,2,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,3,2,3,3,3,3,N,3,3,3,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,4,2,4,4,4,4,N,4,4,4,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,5,2,5,5,5,5,N,5,5,5,98.0,0.0,0.0,0.00,15.50,0.3,113.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,1,99996,99996,99996,99996,N,99996,99996,99996,5.0,0.0,0.5,0.00,0.00,0.3,5.80
99996,99997,1,99997,99997,99997,99997,N,99997,99997,99997,14.0,0.0,0.5,2.00,0.00,0.3,16.80
99997,99998,1,99998,99998,99998,99998,N,99998,99998,99998,29.0,0.0,0.5,8.80,5.54,0.3,44.14
99998,99999,2,99999,99999,99999,99999,N,99999,99999,99999,5.5,0.5,0.5,1.36,0.00,0.3,8.16
